In [ ]:
from bs4 import BeautifulSoup as BS
import requests as req
import pandas as pd
import numpy as npy

In [4]:
file = ("assets/megamillions.csv")

df = pd.read_csv(file, engine='python')

df





,date_code,draw_date,num_1,num_2,num_3,num_4,num_5,MB
0,date=000000000000000000,12/16/2022,8,35,40,53,56,11
1,date=638064864000000000,12/13/2022,14,22,48,58,68,6
2,date=638061408000000000,12/9/2022,8,19,53,61,69,19
3,date=638058816000000000,12/6/2022,15,16,19,28,47,13
4,date=638055360000000000,12/2/2022,1,21,36,46,52,16
...,...,...,...,...,...,...,...,...
1339,date=634018752000000000,2/16/2010,11,12,21,29,45,5
1340,date=634015296000000000,2/12/2010,14,16,17,33,47,23
1341,date=634012704000000000,2/9/2010,2,17,20,26,48,12
1342,date=634009248000000000,2/5/2010,10,20,45,51,53,41
